<img src="https://drive.google.com/uc?export=view&id=1YjAWn06OMcVhlyixBZBDnY17rnn7Otg5" width="100%">

# Arreglos de Dask

En este notebook veremos una introducción práctica al procesamiento distribuido con la librería `dask`, primero lo instalaremos:

In [1]:
!pip install dask[complete] h5py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## **1. ¿Qué son los Arreglos de Dask?**
---

Los arreglos de `dask` son una forma de extender los arreglos de `numpy` para su manejo de forma distribuida y en grandes cantidades de datos. Básicamente es una forma de almacenar un arreglo multidimensional de gran tamaño de forma particionada como múltiples arreglos de `numpy` como se muestra a continuación:

<img src="https://drive.google.com/uc?export=view&id=19tXqc4jTHwcwDUZgz8ARE_hq2uI2gkpT" width="70%%">

Estos arreglos gestionan y coordinan múltiples arreglos de `numpy` (o arreglos bastante compatibles con `numpy`) que pueden existir en distintas máquinas o persistidos en disco.

Desde `dask` tenemos algunas de las operaciones típicas que normalmente usamos con `numpy` como:

* Operaciones aritméticas.
* Operaciones de reducción sobre ejes específicos.
* Reglas de broadcasting como en `numpy`.
* Reordenamiento de ejes.
* Slicing para selección de valores.
* Algunas funciones típicas de álgebra lineal.

No obstante, hay diversas operaciones de `numpy` que no son posibles en arreglos distribuidos dada su naturaleza (por ejemplo, ordenamientos, conversión a listas de _Python_, iteración con ciclos `for`, entre otras).

Veamos las características generales de los arreglos de `dask`. Primero lo importamos:

In [2]:
import dask.array as da
import numpy as np

## **2. Creación**
---

Existen distintas funciones para la creación de arreglos de `dask`, estas las podemos dividir en tres categorías: arreglos determinísticos nativos, arreglos aleatorios nativos y arreglos externos.

### **2.1. Arreglos Determinísticos Nativos**
---

Existen dintintas funciones para la creación de arreglos de `dask` que son similares a `numpy`, por ejemplo:

* `zeros`: crea un arreglo lleno de ceros de un tamaño dado:

In [3]:
X = da.zeros((10, 10), dtype=np.float32)
print(X)

dask.array<zeros_like, shape=(10, 10), dtype=float32, chunksize=(10, 10), chunktype=numpy.ndarray>


* `ones`: crea un arreglo lleno de unos de un tamaño dado:

In [4]:
X = da.ones((10, 10), dtype=np.float32)
print(X)

dask.array<ones_like, shape=(10, 10), dtype=float32, chunksize=(10, 10), chunktype=numpy.ndarray>


* `eye`: crea un arreglo lleno de matrices identidad:

In [5]:
X = da.eye(5, dtype=np.float32)
print(X)

dask.array<eye, shape=(5, 5), dtype=float32, chunksize=(5, 5), chunktype=numpy.ndarray>


* `arange`: crea un arreglo con los valores en un rango dado:

In [6]:
x = da.arange(1, 5)
print(x)

dask.array<arange, shape=(4,), dtype=int64, chunksize=(4,), chunktype=numpy.ndarray>


* `linspace`: crea un número dado de elementos entre dos valores:

In [7]:
x = da.linspace(-1, 1, 100)
print(x)

dask.array<linspace, shape=(100,), dtype=float64, chunksize=(100,), chunktype=numpy.ndarray>


### **2.2. Arreglos Aleatorios Nativos**
---

Al igual que `numpy` tenemos un paquete `random` que nos permite generar arreglos de forma aleatoria, por ejemplo, la función `randint` genera un arreglo con valores enteros asignados de forma aleatoria entre dos números y con un tamaño dado:

In [8]:
X = da.random.randint(1, 5, size=(10, 10))
print(X)

dask.array<randint, shape=(10, 10), dtype=int64, chunksize=(10, 10), chunktype=numpy.ndarray>


De la misma forma, podemos generar números con una distribución uniforme:

In [9]:
X = da.random.uniform(-1, 1, size=(100, 100))
print(X)

dask.array<uniform, shape=(100, 100), dtype=float64, chunksize=(100, 100), chunktype=numpy.ndarray>


Incluso arreglos con distribución normal:

In [10]:
X = da.random.normal(loc=0, scale=1, size=(100, 100))
print(X)

dask.array<normal, shape=(100, 100), dtype=float64, chunksize=(100, 100), chunktype=numpy.ndarray>


Puede usar otras distribuciones aleatorias de `dask` como: `binomial`, `pareto`, `geometric` y muchas más, recuerde que son exactamente las mismas que están en `numpy` y las puede consultar en [este enlace](https://numpy.org/doc/stable/reference/random/index.html).

### **2.3. Arreglos Externos**
---

Podemos crear arreglos de `dask` desde distintas fuentes (como `numpy`), esto se consigue usando la función `from_array` de `dask`. Veamos un ejemplo:

In [11]:
X = da.from_array(np.random.normal(size=(100, 10)), chunks=(10, 10))
print(X)

dask.array<array, shape=(100, 10), dtype=float64, chunksize=(10, 10), chunktype=numpy.ndarray>


En este caso, usamos el parámetro `chunks` para especificar cómo serán internamente las particiones del arreglo. Recuerde que los arreglos de `dask` no están directamente en la memoria RAM hasta que no se especifique, de hecho, todos los arreglos que hemos creado hasta este momento son elementos conocidos como **promesas** (son valores que aún no se ejecutan pero esperamos que sigan determinados criterios como tipo, forma, tamaño, enter otras).

Veamos cuál es el porcentaje de utilización de la memoria RAM antes de crear un arreglo de `dask` con la librería `psutil`:

In [12]:
import psutil
print(f"RAM: {psutil.virtual_memory()[2]}%")

RAM: 7.4%


Ahora, creamos un arreglo de `dask` de un gran tamaño:

In [13]:
X = da.random.normal(size=(1_000_000, 100), chunks=(1000, 100))
print(X)

dask.array<normal, shape=(1000000, 100), dtype=float64, chunksize=(1000, 100), chunktype=numpy.ndarray>


Veamos el porcentaje de utilización de la memoria RAM luego de crear el arreglo:

In [14]:
print(f"RAM: {psutil.virtual_memory()[2]}%")

RAM: 7.4%


Como puede ver, el cambio en la memoria es mínimo (el arreglo no se ha calculado aún y tampoco existe en la memoria).

Un ejemplo típico para el manejo de grandes cantidades de datos desde `dask` es con el formato `hdf5`. Se trata de un formato de archivo que es bastante compacto y que facilita la **lectura aleatoria** (no necesitamos cargar todo el archivo para extraer un valor, podemos hacerlo directamente). Podemos exportar el arreglo directamente al disco con el método `to_hdf5` sin la necesidad de que el archivo esté en la memoria:

In [15]:
X.to_hdf5("array.h5", "/data")

Corroboremos la memoria RAM:

In [16]:
print(f"RAM: {psutil.virtual_memory()[2]}%")

RAM: 7.8%


En este caso, `dask` trabaja únicamente sobre _chunks_ y no sobre el arreglo completo, por ello, únicamente es necesario tener en memoria _chunks_ de tamaño `(1000, 100)` en lugar de un arreglo de tamaño `(1000000, 100)`.

Veamos que se creó un archivo con los datos del arreglo masivo de `dask`:

In [17]:
import os
size = os.stat("array.h5").st_size / (1024 ** 2)
print(f"Tamaño del archivo {size:.2f}MB")

Tamaño del archivo 762.99MB


De la misma forma, podemos cargar un arreglo masivo a partir de archivos `hdf5` desde `dask` con la librería `h5py`:

In [18]:
import h5py
file = h5py.File("array.h5", "r")
data = file["data"]
print(data)

<HDF5 dataset "data": shape (1000000, 100), type "<f8">


Lo cargamos con `dask`:

In [19]:
X = da.from_array(data, chunks=(1000, 100))
print(X)

dask.array<array, shape=(1000000, 100), dtype=float64, chunksize=(1000, 100), chunktype=numpy.ndarray>


Finalmente, cerramos el archivo:

In [20]:
file.close()

## **3. Atributos**
---

Los arreglos de `dask` tienen atributos adicionales a los arreglos de `numpy`. Veamos los más útiles:

El atributo `shape` nos permite extraer el tamaño del arreglo:

In [21]:
print(X.shape)

(1000000, 100)


El atributo `dtype` nos permite saber qué tipo de datos contiene el arreglo:

In [22]:
print(X.dtype)

float64


El atributo `chunksize` permite extraer el tamaño de las particiones del arreglo:

In [23]:
print(X.chunksize)

(1000, 100)


También podemos extraer el número completo de elementos que hay en el arreglo con el atributo `size`:

In [24]:
print(X.size)

100000000


## **4. Métodos**
---

Los arreglos de `dask` usan métodos similares a los de un arreglo de `numpy` y algunos adicionales que nos van a permitir controlar el manejo de memoria. Veamos algunos ejemplos:

* `compute`: este método permite evaluar el arreglo de `dask` y obtener el arreglo de `numpy` correspondiente, veamos un ejemplo:

In [25]:
X = da.random.uniform(size=(100, 10), chunks=(10, 10))
print(X)

dask.array<uniform, shape=(100, 10), dtype=float64, chunksize=(10, 10), chunktype=numpy.ndarray>


Usamos el método `compute`:

In [26]:
X_np = X.compute()
print(X_np)

[[1.78103446e-01 6.54365365e-01 9.32228931e-01 3.21696852e-01
  4.84315667e-02 6.53032014e-01 5.56364408e-01 3.00243492e-01
  5.09839659e-01 8.40208002e-01]
 [9.82101962e-01 2.17570258e-01 7.29214224e-01 9.71354999e-01
  4.54950357e-02 4.24393319e-01 4.65575042e-01 4.59320496e-01
  2.05565736e-01 7.30483453e-01]
 [1.08194862e-01 1.08309860e-01 1.70253703e-01 2.05934058e-01
  2.94290733e-01 2.96998171e-01 1.57332663e-01 1.18079843e-01
  8.00289761e-01 5.56809694e-01]
 [8.33551176e-01 1.30866350e-01 7.13339007e-01 5.78400980e-01
  1.17381448e-01 7.45068806e-01 7.01658660e-01 2.08702823e-01
  9.80902305e-01 5.65736377e-01]
 [1.31759887e-01 3.88063448e-01 3.99986325e-01 1.54405809e-01
  2.61950376e-01 8.19703153e-01 7.41050826e-02 9.84568140e-01
  3.63071829e-01 5.48533603e-01]
 [8.30156704e-01 6.11579867e-01 5.60580815e-01 4.43253504e-01
  8.27970658e-01 5.23234875e-01 8.20525537e-01 6.79650605e-01
  9.90549238e-01 4.38306650e-01]
 [7.05861217e-01 1.29940778e-02 9.49698226e-01 7.08748293e

* `rechunk`: permite cambiar el tamaño de los _chunks_ de un arreglo, por ejemplo:

In [27]:
X2 = X.rechunk(chunks=(50, 10))
print(X2)

dask.array<rechunk-merge, shape=(100, 10), dtype=float64, chunksize=(50, 10), chunktype=numpy.ndarray>


* `persist`: permite guardar en disco los resultados de una operación:

In [28]:
X3 = X2.persist()
print(X3)

dask.array<rechunk-merge, shape=(100, 10), dtype=float64, chunksize=(50, 10), chunktype=numpy.ndarray>


Adicional a esto, tenemos métodos similares como los que manejan los arreglos de `numpy`. Veamos algunos:

* `sum` permite sumar los elementos de un arreglo sobre un eje dado:

In [29]:
sums = X3.sum(axis=1)
print(sums)

dask.array<sum-aggregate, shape=(100,), dtype=float64, chunksize=(50,), chunktype=numpy.ndarray>


Evaluamos el resultado:

In [30]:
print(sums.compute())

[4.99451374 5.23107452 2.81649335 5.57560793 4.12614765 6.72580845
 5.03844985 5.52774664 5.06179262 4.61213427 4.46896826 4.06239599
 5.06269594 4.94438235 5.9700415  5.11619517 4.10986061 4.52105955
 3.68981527 6.79374746 5.44649362 5.35217685 5.84939564 4.44480337
 6.40367939 6.55018291 5.83324791 5.12456203 4.46568744 4.75873532
 3.50377585 5.24961521 3.08261418 5.44096943 5.32115864 5.45494086
 5.84099225 7.34450234 5.10268315 3.70591292 5.94065358 6.41409601
 4.14017107 5.15961078 4.12445205 5.19823892 2.82350072 4.14990631
 6.11028473 4.16229378 4.71405464 6.18717605 5.45157807 4.87557992
 3.56317667 4.31526564 4.27942264 5.55168918 6.16513044 5.47671778
 5.27805917 6.51025286 3.9956978  4.59874982 5.63913547 4.96926844
 4.63877858 5.4139141  6.7194504  3.98639439 2.70168088 3.47652807
 4.44718268 4.917377   5.14012578 3.67603143 5.59602927 4.2561529
 4.88921082 3.30388267 3.65769072 6.52045573 5.1985913  4.0278748
 5.69703809 4.46225498 5.78035977 5.50550455 3.1495322  4.272111

* `mean` permite promediar los elementos de un arreglo sobre un eje dado:

In [31]:
means = X3.mean(axis=1)
print(means)

dask.array<mean_agg-aggregate, shape=(100,), dtype=float64, chunksize=(50,), chunktype=numpy.ndarray>


Evaluamos el resultado:

In [32]:
print(means.compute())

[0.49945137 0.52310745 0.28164933 0.55756079 0.41261477 0.67258085
 0.50384498 0.55277466 0.50617926 0.46121343 0.44689683 0.4062396
 0.50626959 0.49443824 0.59700415 0.51161952 0.41098606 0.45210595
 0.36898153 0.67937475 0.54464936 0.53521768 0.58493956 0.44448034
 0.64036794 0.65501829 0.58332479 0.5124562  0.44656874 0.47587353
 0.35037758 0.52496152 0.30826142 0.54409694 0.53211586 0.54549409
 0.58409923 0.73445023 0.51026832 0.37059129 0.59406536 0.6414096
 0.41401711 0.51596108 0.4124452  0.51982389 0.28235007 0.41499063
 0.61102847 0.41622938 0.47140546 0.61871761 0.54515781 0.48755799
 0.35631767 0.43152656 0.42794226 0.55516892 0.61651304 0.54767178
 0.52780592 0.65102529 0.39956978 0.45987498 0.56391355 0.49692684
 0.46387786 0.54139141 0.67194504 0.39863944 0.27016809 0.34765281
 0.44471827 0.4917377  0.51401258 0.36760314 0.55960293 0.42561529
 0.48892108 0.33038827 0.36576907 0.65204557 0.51985913 0.40278748
 0.56970381 0.4462255  0.57803598 0.55055046 0.31495322 0.427211

* `std` permite calcular la desviación estándar de los elementos de un arreglo sobre un eje dado:

In [33]:
stds = X3.std(axis=1)
print(stds)

dask.array<_sqrt, shape=(100,), dtype=float64, chunksize=(50,), chunktype=numpy.ndarray>


Evaluamos el resultado:

In [34]:
print(stds.compute())

[0.27061184 0.30602438 0.21577325 0.28872272 0.28237956 0.17834305
 0.34935696 0.25650895 0.19754717 0.31285786 0.31810485 0.28662723
 0.32200638 0.27660541 0.25745496 0.25154554 0.27789746 0.30460322
 0.2527053  0.20706052 0.30139998 0.31471161 0.30841514 0.25203208
 0.2298882  0.2291621  0.3072389  0.33116601 0.24340841 0.28319852
 0.30389851 0.23727083 0.27776686 0.2552415  0.2795073  0.26399216
 0.16642848 0.17376342 0.30353251 0.24780797 0.24138559 0.21678812
 0.27136301 0.25664292 0.22873286 0.32641985 0.25753003 0.31968394
 0.2547591  0.30296402 0.24604663 0.28807753 0.27753352 0.26787597
 0.2835618  0.23342439 0.34558013 0.33051858 0.31710905 0.28927591
 0.30601796 0.32846303 0.2130807  0.30804618 0.34235041 0.21511124
 0.27006868 0.27532248 0.22368334 0.24423655 0.19380056 0.30054936
 0.32716376 0.24726117 0.30762153 0.27564691 0.19491204 0.27376995
 0.19959359 0.24276673 0.29224309 0.23379347 0.26566337 0.31306604
 0.28147635 0.31813715 0.27639261 0.31407988 0.19972758 0.2484

* `reshape` premite cambiar la forma de un arreglo:

In [35]:
X4 = X3.reshape((10, 10, 10))
print(X4)

dask.array<reshape, shape=(10, 10, 10), dtype=float64, chunksize=(5, 10, 10), chunktype=numpy.ndarray>


Debe tener cuidado con la operación de `reshape` ya que esta únicamente permite dividir o combinar ejes.

También podemos aplicar la operación `flatten` para cambiar la forma del arreglo

In [36]:
X5 = X4.flatten()
print(X5)

dask.array<reshape, shape=(1000,), dtype=float64, chunksize=(500,), chunktype=numpy.ndarray>


## **5. Operaciones con Arreglos**
---

Podemos realizar distintos tipos de operaciones aritméticas, geométricas y algebraicas sobre los arreglos de `dask`.

Por ejemplo, podemos sumar dos arreglos. Comenzamos definiéndolos:

In [37]:
X1 = da.random.normal(loc=0, scale=1, size=(10, 10))
print(X1)

dask.array<normal, shape=(10, 10), dtype=float64, chunksize=(10, 10), chunktype=numpy.ndarray>


In [38]:
X2 = da.random.normal(loc=0, scale=1, size=(10, 10))
print(X2)

dask.array<normal, shape=(10, 10), dtype=float64, chunksize=(10, 10), chunktype=numpy.ndarray>


La suma se realiza elemento a elemento:

In [39]:
X3 = X1 + X2
print(X3)

dask.array<add, shape=(10, 10), dtype=float64, chunksize=(10, 10), chunktype=numpy.ndarray>


Veamos el resultado:

In [40]:
print(X3.compute())

[[ 0.68718028 -1.39898187 -1.40743384  0.53872001  0.49122893 -3.19766086
   0.79711023 -0.99614257  0.26475617 -0.05668593]
 [ 3.43894017 -0.15804836 -0.72936324 -0.34507115 -1.78216387 -0.61389442
  -0.9617481  -0.17431642  0.82636869  0.44727359]
 [-0.74499136 -2.63171997  1.83668791  2.80137419  0.32338089  0.31473082
   1.80286088  0.27397515 -0.27281019 -0.86082465]
 [ 4.54034766 -1.64492501  0.89890019  2.76780081  0.3638191   0.21915001
   0.39347445  0.11511686 -0.26452376 -0.20146428]
 [-1.13235385 -0.50649444 -1.91987436 -2.31249142 -0.86224844  1.56482571
   0.31132986 -0.17470609 -0.4942202  -2.36558781]
 [-0.62501835  2.14520502 -0.81015614  2.90059623 -0.36973603  1.98439354
   1.32031015  1.23128969  0.74993485  2.4097719 ]
 [ 0.93821586 -0.96562006  0.28699225  0.37467266  0.31579697 -2.11633466
  -2.67881075 -1.0564935  -0.23443608 -0.44416669]
 [-0.64282865 -1.11695377  0.92106077 -2.51956045  1.01489303  1.75379821
  -1.30201811 -2.9979954  -0.64262685  0.04830209]


Lo mismo aplica para operaciones como resta:

In [41]:
X3 = X1 - X2
print(X3)

dask.array<sub, shape=(10, 10), dtype=float64, chunksize=(10, 10), chunktype=numpy.ndarray>


La multiplicación elemento a elemento:

In [42]:
X3 = X1 * X2
print(X3)

dask.array<mul, shape=(10, 10), dtype=float64, chunksize=(10, 10), chunktype=numpy.ndarray>


La división:

In [43]:
X3 = X1 / X2
print(X3)

dask.array<truediv, shape=(10, 10), dtype=float64, chunksize=(10, 10), chunktype=numpy.ndarray>


Podemos realizar un producto matricial con el operador `@`:

In [44]:
X3 = X1 @ X2
print(X3)

dask.array<getitem, shape=(10, 10), dtype=float64, chunksize=(10, 10), chunktype=numpy.ndarray>


También podemos aplicar las reglas de _broadcasting_ como se usan en `numpy`, por ejemplo, para convertir a ceros todas las columnas pares:

In [45]:
mask = (da.arange(10) % 2).reshape(-1, 1)
print(mask)

dask.array<reshape, shape=(10, 1), dtype=int64, chunksize=(10, 1), chunktype=numpy.ndarray>


Aplicamos un producto elemento a elemento para enmascarar:

In [46]:
res = X1 * mask
print(res)

dask.array<mul, shape=(10, 10), dtype=float64, chunksize=(10, 10), chunktype=numpy.ndarray>


Veamos el resultado:

In [47]:
print(res.compute())

[[ 0.         -0.         -0.         -0.          0.         -0.
  -0.         -0.         -0.          0.        ]
 [ 2.52917949  0.49242633  0.55837729 -0.77447491 -2.06587901 -0.92985983
  -0.91721007  0.28146183  0.41293422 -0.68930724]
 [ 0.         -0.          0.          0.         -0.          0.
   0.          0.         -0.         -0.        ]
 [ 2.01070077 -1.07647519  0.31060821  0.83738643 -0.75693682 -1.35016543
   0.46406931 -0.69810097  0.39867365 -1.02105712]
 [-0.         -0.         -0.         -0.         -0.          0.
   0.          0.         -0.         -0.        ]
 [ 0.02156277  0.38978868 -0.69980801  2.29070931  0.10496618  0.05853258
   0.21705232 -0.30659874 -0.14107982  1.09162419]
 [-0.         -0.         -0.          0.          0.         -0.
  -0.          0.         -0.         -0.        ]
 [ 0.13357281 -1.87502072  0.17828243 -0.59993277 -0.03561433  0.24389048
  -0.68582455 -2.54936814 -1.36137255  0.33429571]
 [ 0.          0.         -0.   

También podemos aplicar distintas funciones de forma distribuida de `dask`, por ejemplo la raíz cuadrada:

In [48]:
X3 = da.sqrt(X1)
print(X3)

dask.array<sqrt, shape=(10, 10), dtype=float64, chunksize=(10, 10), chunktype=numpy.ndarray>


El exponente:

In [49]:
X3 = da.exp(X1)
print(X3)

dask.array<exp, shape=(10, 10), dtype=float64, chunksize=(10, 10), chunktype=numpy.ndarray>


El logaritmo:

In [50]:
X3 = da.log(X1)
print(X3)

dask.array<log, shape=(10, 10), dtype=float64, chunksize=(10, 10), chunktype=numpy.ndarray>


El módulo `linalg` de `dask` nos permite realizar algunas operaciones algebraicas que pueden ser distribuidas facilmente, entre ellas tenemos:

* Inverso de una matriz con `inv`.
* Métodos de descomposición matricial como `cholesky`, `svd`, `lu`, `qr`, entre otras.
* Normas de vectores `norm`.

Veamos un ejemplo donde calculamos una matriz al cuadrado, calculamos el inversode la matriz y calculamos la norma de sus vectores con respecto a su primer eje:

In [51]:
X = da.random.normal(size=(10, 5))
X2 = X @ X.T # .T nos permite transponer la matriz.
X3 = da.linalg.inv(X2)
X4 = da.linalg.norm(X3, axis=0)
print(X4.compute())

[2.53897696e+15 3.24310615e+15 4.29218455e+15 1.06144790e+16
 1.00388782e+16 2.26119173e+15 1.37883204e+15 1.08216317e+15
 3.41190469e+15 8.88569489e+15]


Podemos concatenar arreglos de la misma forma que en un arreglo de `numpy` con la función `concatenate`, veamos un ejemplo con los siguientes arreglos:

In [52]:
X1 = da.random.normal(size=(20, 10), chunks=(10, 10))
print(X1)

dask.array<normal, shape=(20, 10), dtype=float64, chunksize=(10, 10), chunktype=numpy.ndarray>


In [53]:
X2 = da.random.normal(size=(20, 20), chunks=(10, 20))
print(X2)

dask.array<normal, shape=(20, 20), dtype=float64, chunksize=(10, 20), chunktype=numpy.ndarray>


Veamos la concatenación:

In [54]:
X3 = da.concatenate([X1, X2], axis=1)
print(X3)

dask.array<concatenate, shape=(20, 30), dtype=float64, chunksize=(10, 20), chunktype=numpy.ndarray>


## **6. Indexación**
---

Podemos indexar los arreglos de `dask` siguiendo la misma notación que usamos en los arreglos de `numpy`, veamos un ejemplo sobre el siguiente arreglo:

In [55]:
X = da.random.normal(size=(100, 10), chunks=(10, 10))
print(X)

dask.array<normal, shape=(100, 10), dtype=float64, chunksize=(10, 10), chunktype=numpy.ndarray>


Podemos seleccionar la primer fila:

In [56]:
x_0 = X[0]
print(x_0)

dask.array<getitem, shape=(10,), dtype=float64, chunksize=(10,), chunktype=numpy.ndarray>


Con `:` podemos seleccionar la primer columna:

In [57]:
x_0 = X[:, 0]
print(x_0)

dask.array<getitem, shape=(100,), dtype=float64, chunksize=(10,), chunktype=numpy.ndarray>


También podemos seleccionar rangos valores específicos, por ejemplo, las dos primeras filas y las dos primeras columnas:

In [58]:
X2 = X[:2, :2]
print(X2)

dask.array<getitem, shape=(2, 2), dtype=float64, chunksize=(2, 2), chunktype=numpy.ndarray>


De igual forma, podemos seleccionar valores específicos dando una lista de valores, por ejemplo, seleccionamos las filas 0, 3 y 5:

In [59]:
X2 = X[[0, 3, 5]]
print(X2)

dask.array<getitem, shape=(3, 10), dtype=float64, chunksize=(3, 10), chunktype=numpy.ndarray>


También podemos aplicar selección condicional:

In [60]:
X2 = X[X > 0]
print(X2)

dask.array<getitem, shape=(nan,), dtype=float64, chunksize=(nan,), chunktype=numpy.ndarray>


Evaluemos el resultado:

In [61]:
print(X2.compute())

[8.27897574e-01 4.59375002e-01 6.53593108e-01 6.06529449e-01
 8.85215768e-01 1.58955368e-01 9.23577241e-01 3.82364144e-01
 1.70869044e+00 3.92238116e-01 1.14270021e+00 6.92450891e-01
 1.75088683e+00 1.77755031e+00 1.26611000e+00 3.52882038e-01
 1.90295508e-02 1.86454120e+00 1.02777146e-01 6.26954935e-01
 1.05801868e+00 4.50862525e-01 8.91626198e-01 1.02083512e-01
 2.28867564e+00 2.64505322e+00 1.65167908e+00 2.07946024e+00
 2.30293137e-01 1.26860883e+00 7.29012060e-01 3.05736853e-01
 6.17890042e-01 2.75712374e-01 1.46474413e+00 9.58341712e-01
 2.01488923e-01 1.28926319e+00 3.32372883e-01 7.57982451e-01
 8.51627570e-01 4.44994095e-01 1.09683472e-01 8.20535054e-01
 5.44228405e-01 1.14757467e+00 2.29952747e-01 8.99396582e-01
 8.01394975e-02 1.50949129e+00 1.07816775e+00 1.69651392e-01
 6.30205547e-01 1.07104684e+00 1.16069669e+00 3.81233265e-01
 1.59113641e+00 1.10172750e+00 1.29314765e+00 2.85604324e-01
 2.48559556e+00 1.02986004e+00 2.00645704e-01 1.49490780e+00
 5.70440661e-01 1.185399

Como puede ver, los arreglos de `dask` son una herramienta poderosa que nos permite escalar los arreglos de `numpy` a grandes cantidades de datos y su ejecución de forma distribuida. No obstante, si estamos manejando un conjunto de datos pequeño no es muy recomendable usar `dask`, veamos una comparativa entre las operaciones con `dask` y con `numpy` con una matriz pequeña de `(1000, 1000)` usando la librería `time`:

In [62]:
import time

Vamos a implementar un producto matricial desde `numpy` primero y a evaluar el tiempo:

In [63]:
t0 = time.time()
X = np.random.uniform(size=(1000, 1000))
Y = X @ X
delta_t = time.time() - t0
print(delta_t)

0.09443092346191406


Ahora veamos la misma operación con `dask`:

In [64]:
t0 = time.time()
X = da.random.uniform(size=(1000, 1000), chunks=(100, 100))
Y = (X @ X).compute()
delta_t = time.time() - t0
print(delta_t)

/usr/local/lib/python3.7/dist-packages/dask/array/routines.py:435: PerformanceWarning: Increasing number of chunks by factor of 10
  concatenate=False,


0.7282745838165283


Como puede ver, la operación con `dask` puede tomar más tiempo. Esto se debe a que `dask` tiene que coordinar y manipular las operaciones entre varios arreglos y luego unir los resultados. Esto tiene beneficios con grandes cantidades de datos pero genera un cuello de botella con datasets pequeños.

## **7. Recursos Adicionales**
---

* [Arreglos de Dask](https://docs.dask.org/en/stable/array.html).
* [Dask - Talks & tutorials](https://docs.dask.org/en/stable/presentations.html).

## **8. Créditos**
---

**Profesor**

- [Jorge E. Camargo, PhD](https://dis.unal.edu.co/~jecamargom/)

**Diseño, desarrollo del notebook y material audiovisual**

- [Juan S. Lara MSc](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/)

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*